# Creating and modifying Topologies

PyGromosTools offers a wide variaty of tools and functions to assist in the creation and modification of specialized topologies.

In the following notebook we will demonstrate some of the functions and theire usage. These functions and workflows should work with all supported forcfields, but we will use OpenForceField due to the fact that it is independant of make_top and a Gromos++ installation.

In [1]:
import os
from rdkit.Chem import AllChem

In [2]:
from pygromos.files.gromos_system.gromos_system import Gromos_System
from pygromos.files.forcefield.openff import openff
from pygromos.files.topology.top import Top

In [3]:
# setting the work directory
root_dir = os.path.abspath("../example_files/topo_tutorial/")

## Automatic System generation and combining of topologies

In the first part of this notebbok we demonstrate some automatic tools to create and modifie topologies.

In [4]:
# defining the forcefield we want to use. In case a Gromos system is used make sure to have a valid Gromos++ installation and define the gromos++ path in Gromos_System or add it to you Path.
ff = openff.OpenFF()

Found off: /home/mlehner/PyGromosTools/pygromos/data/ff/SMIRNOFF/openff_unconstrained-2.0.0.offxml


In [5]:
# We define a molecule in RDKit. Note that this molecule could be modified in RDKit or read from sdf, pdb, ... files
mol = AllChem.MolFromSmiles("C1CCCCC1")
mol = AllChem.AddHs(mol)
AllChem.EmbedMolecule(mol)


0

In [6]:
# We define our Gromos_system. Note the option auto_convert=True to automatically genarte a valid topology from the molecule defined in mol and the forcefield defined in ff.
grosys = Gromos_System(work_folder=root_dir, system_name="test", forcefield=ff, 
                       rdkitMol=mol, auto_convert=True)

Found off: /home/mlehner/PyGromosTools/pygromos/data/ff/SMIRNOFF/openff_unconstrained-2.0.0.offxml


In [7]:
# The topology created in the last step printed
grosys.top.TITLE

TITLE

	name: 	 hill_formula: C6H12
	----------------------------------------
	| created from OpenForceField topology |
	| use Amber Block for OpenFF topology! |
	----------------------------------------

	>>> Generated with PyGromosTools (riniker group) <<<
END

After the automatic creation of this topology for a Cyclohexan molecule we could try to make a liquid topology. Therefore we want to have multiple copies of our topology in one single topology.

In [8]:
# define a new, empty top
retTop=Top(in_value=None)

With the "+=" operator we can add multiple topologies to a single topology. Or multiply the topology in a for loop. This alows costum topologies for liquids. After the loop we can reasign the variable to our Gromos_System and check if the additional molecules were added.

In [9]:
for _ in range(10):
    retTop += grosys.top
grosys.top = retTop

In [10]:
# we added 10 copies of the initial moleucle to retTop and our new topology now contains 10 molecules in the RESNAME block.
grosys.top.RESNAME

RESNAME
10
C6H12
C6H12
C6H12
C6H12
C6H12
C6H12
C6H12
C6H12
C6H12
C6H12
END

In [11]:
grosys.top.make_ordered()

We can also add different molecules in our system. For example we could add a Toluene to our Cyclohexane system.

In [12]:
# create a topology for Toluene
tolTop = Gromos_System(work_folder=root_dir, system_name="TOL", in_smiles="c1ccccc1C", forcefield=ff, auto_convert=True).top

Found off: /home/mlehner/PyGromosTools/pygromos/data/ff/SMIRNOFF/openff_unconstrained-2.0.0.offxml


In [13]:
# add the toluene topology to our main topology
grosys.top += tolTop

In [14]:
tolTop.LJPARAMETERS.content[0].C6

0.0025854006549356347

In [15]:
# check if it was added properly
grosys.top.RESNAME

RESNAME
11
C6H12
C6H12
C6H12
C6H12
C6H12
C6H12
C6H12
C6H12
C6H12
C6H12
C7H8
END

If many topologies should be added to a single topology the PyGromosTools version of com_top can be used which directly implements multipliers. Instead of for loops.

## Manual topology modifications

In this second part of the notebook we demonstrate some functions to manually tweek topologies.

Like with any othe PyGromosTools file class we have the option to brute force rewrite blocks. However, this often requires advanced knowlede of the block structure. Except for a few simple cases like the TITLE block.

In [16]:
grosys.top.add_block(blocktitle="TITLE", content=["This is a new Title"])

In [17]:
grosys.top.make_ordered()

In [18]:
grosys.top.TITLE

TITLE
This is a new Title
	>>> Generated with PyGromosTools (riniker group) <<<
END

For more complexe blocks PyGromosTools provides many functions to modify topologies. For example we could simply add a new soluteatom to the system. This would be relevant if you want to manually add a ion with very specific Nonbonded values to you system. These tools are vary powrful, but don't do any sanity checks on the values or compatibility to your forcefields.

In [19]:
# We add a ion named 'XX' with charge +1 and certain LJ-Parameters to the main topology
grosys.top.add_new_atom(PANM="XX", CG=1, C6=1e-3, C12=1e-6, verbose=True)

New Atomtype with LJ parameters added. IAC found as: 5


In [20]:
# Check for the new Atomtype 'XX'
grosys.top.ATOMTYPENAME

ATOMTYPENAME
5
n16
n2
n14
n7
XX
END

In [21]:
grosys.top.SOLUTEATOM

SOLUTEATOM
#   NRP: number of solute atoms
 	 196
#  ATNM: atom number
#  MRES: residue number
#  PANM: atom name of solute atom
#   IAC: integer (van der Waals) atom type code
#  MASS: mass of solute atom
#    CG: charge of solute atom
#   CGC: charge group code (0 or 1)
#   INE: number of excluded atoms
# INE14: number of 1-4 interactions
# ATNM MRES PANM IAC     MASS       CG  CGC INE
#                                           INE14
	1	1	C1	1	12.01078	-0.0764	0	10	2	3	5	6	7	8
											9	10	17	18
										5	4	11	12	15	16
	2	1	C2	1	12.01078	-0.0764	0	9	3	4	6	7	8	9
											10	11	12
										5	5	13	14	17	18
	3	1	C3	1	12.01078	-0.0764	0	8	4	5	9	10	11	12
											13	14
										5	6	7	8	15	16
	4	1	C4	1	12.01078	-0.0764	0	8	5	6	11	12	13	14
											15	16
										4	9	10	17	18
	5	1	C5	1	12.01078	-0.0764	0	7	6	13	14	15	16	17
											18
										4	7	8	11	12
	6	1	C6	1	12.01078	-0.0764	0	6	7	8	15	16	17	18
										4	9	10	13	14
	7	1	H1	2	1.007947	0.0382	0	1	8
										4	9	10	1

In [22]:
grosys.rebase_files()

File imd is empty , can not be written!


/home/mlehner/PyGromosTools/pygromos/files/gromos_system/gromos_system.py:869: UserWarning: Did not change file path as its only promised None
  warnings.warn("Did not change file path as its only promised " + str(file_obj.path))
